In [ ]:
import torch
import open3d as o3d
import time
import numpy as np
from vggt.models.vggt import VGGT
from modules.aligned_vggt_main_lidar import AlignedVGGTLidar
from modules.utils import *
from modules.input_stream_main import *
from modules.visualizer import *
from modules.semantic_segmentator import *
from modules.instance_tracker import *
from ultralytics import YOLO
from tqdm import tqdm
import copy

from modules.floor_plane_clean import FloorPlaneFinder
from modules.navigation_new import NavMapGenerator
from modules.anna import Anna

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model = VGGT.from_pretrained("facebook/VGGT-1B").to(device)

Using device: cuda


In [ ]:
base_path = "path/to/folder"

In [4]:
num_of_imgs_to_use, step = 1500, 2

In [5]:
input_stream = InputStreamMain(rgb_folder=base_path,rgb_pattern="colorImg*.jpg",depth_folder=base_path,depth_pattern="depthMat*.csv",data_limit=num_of_imgs_to_use,step=step)#

rgb folder is not none, it is:  /media/buvar/HBVCSSD/__DATA__/in-the-wild-PPCU/noFilter_lab240_20260130/2026_01_30_taskname_H0x0yW_frames/


In [6]:
actual_img_size = input_stream.img_size
w, h = input_stream.img_size
fixed_w = 518
vggt_img_size = (fixed_w, int(round(h * (fixed_w / w) / 14) * 14))

In [7]:

vis = o3d.visualization.Visualizer()
vis.create_window()
vis_options = vis.get_render_option()
ctr = vis.get_view_control()
ctr.rotate(90.0, 90.0, 0.0)
ctr.translate(-10.0, 0.0)
vis_options.background_color = np.asarray([0,0,0])
vis_options.point_size = 2.0
# Initiate interactive viewer
vis.clear_geometries()
vis.poll_events()
vis.update_renderer()

In [8]:
yolo_model = YOLO("yolov9e-seg.pt")
type_list = list(range(80))
yolo_model.overrides["classes"] = type_list
type_name_dict = { 0: "person",  1: "bicycle",  2: "car",  3: "motorcycle",  4: "airplane",  5: "bus",  6: "train",  7: "truck",  8: "boat",  9: "traffic light",  10: "fire hydrant",  11: "stop sign",  12: "parking meter",  13: "bench",  14: "bird",  15: "cat",  16: "dog",  17: "horse",  18: "sheep",  19: "cow",  20: "elephant",  21: "bear",  22: "zebra",  23: "giraffe",  24: "backpack",  25: "umbrella",  26: "handbag",  27: "tie",  28: "suitcase",  29: "frisbee",  30: "skis",  31: "snowboard",  32: "sports ball",  33: "kite",  34: "baseball bat",  35: "baseball glove",  36: "skateboard",  37: "surfboard",  38: "tennis racket",  39: "bottle",  40: "wine glass",  41: "cup",  42: "fork",  43: "knife",  44: "spoon",  45: "bowl", 46: "banana",  47: "apple",  48: "sandwich",  49: "orange",  50: "broccoli",  51: "carrot",  52: "hot dog",  53: "pizza",  54: "donut",  55: "cake",  56: "chair",  57: "couch",  58: "potted plant",  59: "bed",  60: "dining table",  61: "toilet",  62: "tv",  63: "laptop",  64: "mouse",  65: "remote",  66: "keyboard",  67: "cell phone",  68: "microwave",  69: "oven",  70: "toaster",  71: "sink",  72: "refrigerator",  73: "book",  74: "clock",  75: "vase",  76: "scissors",  77: "teddy bear",  78: "hair drier",  79: "toothbrush"}

In [9]:
semantic_segmentator = SemanticSegmentator(yolo_model,type_name_dict,type_list,vggt_img_size,conf_threshold=0.4)
instance_tracker = InstanceTracker(vggt_img_size,type_list)
renderer = Renderer()

In [10]:
plane = FloorPlaneFinder()
#change min_height if the floor points are in the nav map
navmap = NavMapGenerator(0.05,min_height=0.2,max_height=2.0,normal_min_height=0.4,normal_threshold=0.8)
plane.it_n = 1000
plane.exp_vector_confusion = False
#plane.d_tr = 0.075
plane.exp_d_adjust = True
plane_updates = []
meanmap_tr = 1
plane_ds_rate = 10
navmap_input_pcd = np.empty((0,3))

def generate_plane_mesh(size,height,color):
    vert = np.array([
        [size,size,0],[-size,size,0],[size,-size,0],[-size,-size,0]
    ])
    tri = np.array([
        [0,2,3],[0,3,1]
    ])
    tnorm = np.array([
        [0,0,-1],[0,0,-1]
    ])
    vnorm = np.array([
        [0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1]
    ])
    plane = o3d.geometry.TriangleMesh.create_box(2*size,2*size,height)
    plane.translate([-size,-size,0])
    plane.paint_uniform_color(color)
    return plane
plane_mesh = generate_plane_mesh(10,1/1000,[0,0.5,0.5])

plane_mesh_rotated = generate_plane_mesh(10,1/1000,[0,0.5,0.5])

def simple_aggregate(per_frame_pcd,ds_rate): # TODO Switch to filtered visualizer fn.
    full_pcd = np.empty((0,3))
    for pcd in per_frame_pcd:
        full_pcd = np.append(full_pcd,np.reshape(pcd,(-1,3))[::ds_rate],0)
    return full_pcd


In [11]:
def vis_map(name,map,cmin=0.0,cmax=2.0):
    r = np.logical_not(np.isnan(map))
    g = np.zeros_like(map)
    if np.any(r):
        g[r] = np.clip(map[r]-cmin,cmin,cmax)/(cmax-cmin)*255

    b = np.zeros_like(map)
    r = r*255
    bgr = np.dstack((b,g,r)).astype(np.uint8)
    cv2.imshow(name,np.flip(bgr,0))

def vis_semantic_map(name,map,block_id):
    cmap = Renderer.get_color_map()
    bgr = np.empty((map.shape[0],map.shape[1],3),dtype=np.uint8)
    for x in range(0,bgr.shape[0]):
        for y in range(0,bgr.shape[1]):
            id = map[x,y]
            if np.isnan(id):
                bgr[x,y,:] = [0,0,0]
            else:
                bgr[x,y,:] = cmap[int(id)]

    cv2.imshow(name,np.flip(bgr,0))
    cv2.imwrite(f"output/navmap-lab-nd/{block_id}.png",np.flip(bgr,0))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
aligned_vggt = AlignedVGGTLidar(model,block_size = 10,keyframe_per_block = 10,keyframe_memory_cap = 10, depth_conf_threshold = 1.1,device = device)
last_full_block = list(range(0,len(input_stream),aligned_vggt.block_size))[-1] # To handle if the frame num is not divisible by the block size
use_semantics = True
use_instance_tracking = True
do_navigation = True
rotated_fully_aggregated_pcd = o3d.geometry.PointCloud()
pathfinder = Anna(vggt=aligned_vggt)
if len(input_stream) % aligned_vggt.block_size != 0:
    last_full_block -= aligned_vggt.block_size
for block_idx in tqdm(range(0,last_full_block+1,aligned_vggt.block_size)):
    aggregated_pcd = o3d.geometry.PointCloud()
    start_time = time.time()
    # Get frames relevant to this block
    frames = input_stream.get_frames(aligned_vggt.block_size)
    if use_instance_tracking == True:
        tracked_frames = aligned_vggt.keyframe_list + frames
        block_indices = list(range(block_idx,int(np.amin([block_idx+aligned_vggt.block_size,len(input_stream)]))))
        tracked_frame_indices = aligned_vggt.keyframe_list_idx + block_indices
    if frames[0] is None:
        break
    # Process new frames and keyframes with VGGT
    print(f"Block {block_idx}")
    images, per_frame_point_clouds, depth_mask = aligned_vggt.process_frames(frames, block_idx)
    
    end_time = time.time()
    
    if use_semantics == True:
        start_time = time.time()
        semantic_segmentator.segment_frames(frames,aligned_vggt,block_idx)
        end_time = time.time()
    if use_instance_tracking == True:
        start_time = time.time()
        instance_tracker.run_instance_tracking(aligned_vggt,semantic_segmentator,tracked_frame_indices)
        end_time = time.time()
    if use_semantics == True and do_navigation == True:
        
        transform_updated = plane.advance(aligned_vggt,semantic_segmentator)
        last_block_pcd = renderer.create_semantic_pcd(aligned_vggt,semantic_segmentator,False,True,downsample_rate=10)
        navmap_input_pcd = np.append(navmap_input_pcd,np.asarray(last_block_pcd.points),0)

        if transform_updated is not plane.STATE_NO_UPDATE:
            plane_updates.append((block_idx,transform_updated))

        if transform_updated != plane.STATE_NO_UPDATE:
            navmap.reset(plane.transform,navmap_input_pcd,semantic_segmentator)
        else:
            navmap.update(plane.transform,np.asarray(last_block_pcd.points),semantic_segmentator)
        
        pathfinder.get_navigation_and_visualize(
            block_idx=block_idx,
            navmap=navmap,
            plane=plane,
            goal_id=1
        )

        vis_map("mean",navmap.get_mean(),0,5)
        vis_map("min",navmap.min)
        vis_map("max",navmap.max)
        vis_semantic_map("semantics",navmap.semantic,block_idx)
        cv2.waitKey(1)
    
    aligned_vggt.clean_memory()
    start_time = time.time()
    if use_semantics == True:
        if do_navigation == True:
            rendered_pcd = last_block_pcd
        else:
            rendered_pcd = renderer.create_semantic_pcd(aligned_vggt,semantic_segmentator,False,True,downsample_rate=10)
    else:
        rendered_pcd = renderer.create_aggregated_pcd(aligned_vggt,True,downsample_rate=10)
    R = rendered_pcd .get_rotation_matrix_from_axis_angle([np.radians(220), 0, 0])
    rendered_pcd .rotate(R, center=(0, 0, 0)) 
    if use_semantics == True and do_navigation == True and transform_updated is not plane.STATE_NO_UPDATE:
        vis.remove_geometry(plane_mesh_rotated)
        plane_mesh_rotated = copy.deepcopy(plane_mesh).transform(plane.inv_transform)
        R = plane_mesh_rotated.get_rotation_matrix_from_axis_angle([np.radians(220), 0, 0])
        plane_mesh_rotated.rotate(R, center=(0, 0, 0)) 
        vis.add_geometry(plane_mesh_rotated)

    if len(rendered_pcd.points) > 0:
        vis.add_geometry(rendered_pcd)
        vis.poll_events()
        vis.update_renderer()
    end_time = time.time()
    print(f"Rendered {len(aligned_vggt.keyframe_list_idx) + len(frames)} frames in {end_time - start_time}s")
vis.run()
vis.destroy_window()
cv2.destroyAllWindows()